In [ ]:
import ROOT
from ROOT import TGraph, TLine, TH1F, TCanvas, TLegend, TPaveText, kRed, kGreen, kMagenta, kBlue
from ROOTDefs import get_signal_and_background_files, get_formatted_root_tree, tau_data_directory
import os
import numpy as np
#from bigClusterHelpers import bigCluster
from myHelpers_New import bigCluster

In [ ]:
#tsig, fsig, tback, fback

run2_tback, run2_fback = get_formatted_root_tree(os.path.join(tau_data_directory(), 'back_ntuple_turnonrunII.root'))
print(run2_tback.entries)
run3_tback, run3_fback = get_formatted_root_tree(os.path.join(tau_data_directory(), 'back_ntuple_turnonrunIII.root'))
print(run3_tback.entries)
run3_tback_weighted, run3_fback_weighted = get_formatted_root_tree(os.path.join(tau_data_directory(), 'back_ntuple_turnonrunIII_weighted.root'))
print(run3_tback_weighted.entries)
bigcluster_tback, bigcluster_fback = get_formatted_root_tree(os.path.join(tau_data_directory(), 'back_ntuple_turnonbigcluster.root'))
print(bigcluster_tback.entries)
run3_cell_tback, run3_cell_fback = get_formatted_root_tree(os.path.join(tau_data_directory(), 'back_cells_runIII.root'))
print(run3_cell_tback.entries)

In [ ]:
%jsroot on
c1 = TCanvas('c1', 'Graph Draw Options', 200, 10, 800, 600)

In [ ]:
event_num = run3_tback.entries

run3_ets = np.zeros(event_num)
bigcluster_ets = np.zeros(event_num)

for i in range(event_num):
    run3_tback.root_ttree.GetEntry(i)
    run3_ets[i] = run3_tback.root_ttree.Run3Et
    
    bigcluster_tback.root_ttree.GetEntry(i)
    bigcluster_ets[i] = bigcluster_tback.root_ttree.BigClusterEt
    
scatterer = TGraph(run3_tback.entries, run3_ets, bigcluster_ets)

liner = TLine(0, 0, 100, 100)

c1.Clear()

scatterer.Draw('ap')
scatterer.SetMarkerColor(kBlue)
scatterer.SetTitle('Oregon vs. bigCluster Et')
scatterer.GetXaxis().SetTitle('Oregon Et')
scatterer.GetYaxis().SetTitle('bigCluster Et')
scatterer.GetXaxis().SetRangeUser(0, 100)
scatterer.GetYaxis().SetRangeUser(0, 100)

liner.Draw('same')

c1.Draw()

In [ ]:
run2_et_hist = TH1F('hist', 'Run-II Background Et', 100, 0, 25)
run2_et_hist.GetXaxis().SetTitle('Reconstructed Et')
run2_et_hist.GetYaxis().SetTitle('Entries')

run2_et_hist.Reset()

runII_counter_20 = 0
runII_counter_12 = 0
runII_counter_sub1 = 0
runII_counter_over12 = 0
runII_counter_over50 = 0

for i in range(run2_tback.entries):
    run2_tback.root_ttree.GetEntry(i)
    et = run2_tback.root_ttree.Run2Et
    
    if et > 12:
        runII_counter_12 += 1
    if et > 20:
        runII_counter_20 += 1
    

#run2_et_hist.Draw()
#c1.Draw()

print(runII_counter_12)
print(runII_counter_20)

In [ ]:
# Run 2 count of events >= 12
#runII_counter_12 = 7990
# Run 2 count of events > 12
runII_counter_12 = 7119

print('Run-II background events over 20 GeV: ',runII_counter_20)
print('Run-II background events over 12 GeV: ',runII_counter_12)

In [ ]:
run3_et_hist = TH1F('hist', 'Run-III Background Et', 100, 0, 25)
run3_et_hist.GetXaxis().SetTitle('Reconstructed Et')
run3_et_hist.GetYaxis().SetTitle('Entries')

run3_et_hist.Reset()

run3_counter_over12 = 0
run3_counter_under0 = 0

for i in range(run3_tback.entries):
    run3_tback.root_ttree.GetEntry(i)
    event_et = run3_tback.root_ttree.Run3Et
    #if i < 1000:
    #    print(event_et)
    if event_et < 0:
        run3_counter_under0 += 1
    if event_et > 12:
        run3_counter_over12 += 1
    run3_et_hist.Fill(event_et)  

c1.Clear()
run3_et_hist.Draw()
c1.Draw()
print(run3_counter_over12)
print(run3_counter_under0)

In [ ]:
bigcluster_et_hist = TH1F('hist', 'Big Cluster Background Et', 100, 0, 25)
bigcluster_et_hist.GetXaxis().SetTitle('Reconstructed Et')
bigcluster_et_hist.GetYaxis().SetTitle('Entries')

bigcluster_et_hist.Reset()

bigcluster_counter_over12 = 0
bigcluster_counter_under0 = 0

for i in range(bigcluster_tback.entries):
    bigcluster_tback.root_ttree.GetEntry(i)
    event_et = bigcluster_tback.root_ttree.BigClusterEt
    #if i < 100:
    #    print(event_et)
    if event_et < 0:
        bigcluster_counter_under0 += 1
    if event_et > 12:
        bigcluster_counter_over12 += 1
    bigcluster_et_hist.Fill(event_et)  
    
c1.Clear()
bigcluster_et_hist.Draw()
c1.Draw()

print(bigcluster_counter_over12)
print(bigcluster_counter_under0)

In [ ]:
runII_lowet_threshold = 12
print(runII_counter_12)

In [ ]:
et_cuts = [ 0.025 * i for i in range(1200)]
events_above_cut = [0]*1200    

runIII_lowet_threshold = None
entries = run3_tback.entries
for j, cut in enumerate(et_cuts):
    if runIII_lowet_threshold is not None:
        break
    for i in range(entries):
        run3_tback.root_ttree.GetEntry(i)
        event_et = run3_tback.root_ttree.Run3Et
        if event_et > cut:
            events_above_cut[j] += 1
    if events_above_cut[j] <= runII_counter_12:
        runIII_lowet_threshold = cut
        break
        
print(runIII_lowet_threshold)

In [ ]:
et_cuts = [ 0.025 * i for i in range(1200)]
events_above_cut = [0]*1200    

runIII_highet_threshold = None
entries = run3_tback.entries
for j, cut in enumerate(et_cuts):
    if runIII_highet_threshold is not None:
        break
    for i in range(entries):
        run3_tback.root_ttree.GetEntry(i)
        event_et = run3_tback.root_ttree.Run3Et
        if event_et > cut:
            events_above_cut[j] += 1
    if events_above_cut[j] <= runII_counter_20:
        runIII_highet_threshold = cut
        break
        
print(runIII_highet_threshold)

In [ ]:
et_cuts = [ 0.025 * i for i in range(1200)]
events_above_cut = [0]*1200    

bigcluster_lowet_threshold = None
entries = bigcluster_tback.entries
for j, cut in enumerate(et_cuts):
    if bigcluster_lowet_threshold is not None:
        break
    for i in range(entries):
        bigcluster_tback.root_ttree.GetEntry(i)
        event_et = bigcluster_tback.root_ttree.BigClusterEt
        if event_et > cut:
            events_above_cut[j] += 1
    if events_above_cut[j] <= runII_counter_12:
        bigcluster_lowet_threshold = cut
        break                  
        
print(bigcluster_lowet_threshold)

In [ ]:
# Values calculated above
runII_lowet_threshold = 12
runIII_lowet_threshold = 13
bigcluster_lowet_threshold = 13.8

In [ ]:
# Modify values for testing
runII_lowet_threshold = 12
runIII_lowet_threshold = 12.5
bigcluster_lowet_threshold = 13.5

In [ ]:
print(tau_data_directory())
run2_tsig, run2_fsig = get_formatted_root_tree(os.path.join(tau_data_directory(), 'sig_ntuple_turnonrunII.root'))
print(run2_tsig.entries)
run3_tsig, run3_fsig = get_formatted_root_tree(os.path.join(tau_data_directory(), 'sig_ntuple_turnonrunIII.root'))
print(run3_tsig.entries)
#run3_tsig_weighted, run3_fsig_weighted = get_formatted_root_tree(os.path.join(tau_data_directory(), 'sig_ntuple_turnonrunIII_weighted.root'))
#print(run3_tsig_weighted.entries)
bigcluster_tsig, bigcluster_fsig = get_formatted_root_tree(os.path.join(tau_data_directory(), 'sig_ntuple_turnonbigcluster.root'))
print(bigcluster_tsig.entries)

In [ ]:
run2_sig_pt = TH1F('hist', 'Run-II Signal Reco Tau Pt', 40, 0, 80)
run2_sig_pt.GetXaxis().SetTitle('Reco Tau Pt')
run2_sig_pt.GetYaxis().SetTitle('Entries')

for i in range(run2_tsig.entries):
    run2_tsig.root_ttree.GetEntry(i)
    run2_sig_pt.Fill(run2_tsig.root_ttree.RecoTauPt)
    
c1.Clear()
run2_sig_pt.Draw()
c1.Draw()

In [ ]:
run2_sig_pt_12gev = TH1F('hist', 'Run-II Signal Reco Tau Pt (Reconstructed Et > {} GeV)'.format(runII_lowet_threshold), 40, 0, 80)
run2_sig_pt_12gev.GetXaxis().SetTitle('Reco Tau Pt')
run2_sig_pt_12gev.GetYaxis().SetTitle('Entries')

for i in range(run2_tsig.entries):
    run2_tsig.root_ttree.GetEntry(i)
    event_et = run2_tsig.root_ttree.Run2Et
    if event_et <= runII_lowet_threshold:
        continue
    run2_sig_pt_12gev.Fill(run2_tsig.root_ttree.RecoTauPt)
    
c1.Clear()
run2_sig_pt_12gev.Draw()
c1.Draw()

In [ ]:
run2_turnon_12gev = run2_sig_pt_12gev.Clone()
run2_turnon_12gev.SetTitle('Run-II Turn-On (Reconstructed Et > {} GeV)'.format(runII_lowet_threshold))


run2_turnon_12gev.Divide(run2_sig_pt)

c1.Clear()
run2_turnon_12gev.Draw()
run2_turnon_12gev.SetStats(0)
c1.Draw() 

In [ ]:
run3_sig_pt = TH1F('hist', 'Run-III Reco Tau Pt', 40, 0, 80)
run3_sig_pt.GetXaxis().SetTitle('Reco Tau Pt')
run3_sig_pt.GetYaxis().SetTitle('Entries')

for i in range(run3_tsig.entries):
    run3_tsig.root_ttree.GetEntry(i)
    run3_sig_pt.Fill(run3_tsig.root_ttree.RecoTauPt)

c1.Clear()
run3_sig_pt.Draw()
c1.Draw()

In [ ]:
run3_sig_pt_low = TH1F('hist', 'Run-III Reco Tau Pt (Reconstructed Et > {} GeV)'.format(runIII_lowet_threshold), 40, 0, 80)
run3_sig_pt_low.GetXaxis().SetTitle('Reco Tau Pt')
run3_sig_pt_low.GetYaxis().SetTitle('Entries')

for i in range(run3_tsig.entries):
    run3_tsig.root_ttree.GetEntry(i)
    if run3_tsig.root_ttree.Run3Et <= runIII_lowet_threshold:
        continue
    run3_sig_pt_low.Fill(run3_tsig.root_ttree.RecoTauPt)

c1.Clear()
run3_sig_pt_low.Draw()
c1.Draw()

In [ ]:
run3_turnon_low = run3_sig_pt_low.Clone()
run3_turnon_low.SetTitle('Run-III Turn-On (Reconstructed Et > {} GeV)'.format(runIII_lowet_threshold))

run3_turnon_low.Divide(run3_sig_pt)

c1.Clear()
run3_turnon_low.Draw()
run3_turnon_low.SetStats(0)
c1.Draw() 

In [ ]:
c1.Clear()
bigcluster_sig_pt = TH1F('hist', 'BigCluster Reco Tau Pt', 40, 0, 80)
bigcluster_sig_pt.GetXaxis().SetTitle('Reco Tau Pt')
bigcluster_sig_pt.GetYaxis().SetTitle('Entries')

for i in range(bigcluster_tsig.entries):
    bigcluster_tsig.root_ttree.GetEntry(i)
    bigcluster_sig_pt.Fill(bigcluster_tsig.root_ttree.RecoTauPt)

bigcluster_sig_pt.Draw()

c1.Draw()

In [ ]:
c1.Clear()
bigcluster_sig_pt_low = TH1F('hist', 'BigCluster Reco Tau Pt (Reconstructed Et > {} GeV)'.format(bigcluster_lowet_threshold), 40, 0, 80)
bigcluster_sig_pt_low.GetXaxis().SetTitle('Reco Tau Pt')
bigcluster_sig_pt_low.GetYaxis().SetTitle('Entries')

for i in range(bigcluster_tsig.entries):
    bigcluster_tsig.root_ttree.GetEntry(i)
    if bigcluster_tsig.root_ttree.BigClusterEt <= bigcluster_lowet_threshold:
        continue
    bigcluster_sig_pt_low.Fill(bigcluster_tsig.root_ttree.RecoTauPt)

bigcluster_sig_pt_low.Draw()

c1.Draw()

In [ ]:
c1.Clear()
bigcluster_turnon_low = bigcluster_sig_pt_low.Clone()
bigcluster_turnon_low.SetTitle('BigCluster Turn-On (Reconstructed Et > {} GeV)'.format(bigcluster_lowet_threshold))

bigcluster_turnon_low.Divide(bigcluster_sig_pt)


bigcluster_turnon_low.Draw()
bigcluster_turnon_low.SetStats(0)
c1.Draw()   

In [ ]:
c1.Clear()
run2_turnon_12gev.Draw('hist')
run2_turnon_12gev.SetStats(0)
run2_turnon_12gev.SetTitle('Turn-On Curves (Fine Thresholds)')
run2_turnon_12gev.GetYaxis().SetTitle('Efficiency')
run2_turnon_12gev.GetYaxis().SetRangeUser(0, 1)
run3_turnon_low.Draw('hsame')
run3_turnon_low.SetLineColor(kRed)
#run3_turnon_low_weighted.Draw('hsame')
#run3_turnon_low_weighted.SetLineColor(kMagenta)
bigcluster_turnon_low.Draw('hsame')
bigcluster_turnon_low.SetLineColor(kGreen)

l1 = TLegend(0.55, 0.1, 0.9, 0.25)
l1.AddEntry(run2_turnon_12gev, 'Run-II @ {} GeV'.format(runII_lowet_threshold))
l1.AddEntry(run3_turnon_low, 'Oregon @ {} GeV'.format(runIII_lowet_threshold))
#l1.AddEntry(run3_turnon_low_weighted, 'Weighted Oregon')
l1.AddEntry(bigcluster_turnon_low, 'BigCluster @ {} GeV'.format(bigcluster_lowet_threshold))
l1.Draw('same')
#c1.Draw()
c1.Print('../../AllTurnOnFineThresholds.pdf')

In [ ]:
run3_tsig_unw, run3_fsig_unw = get_formatted_root_tree(os.path.join(tau_data_directory(), 'sig_ntuple_turnonrunIII.root'))
print(run3_tsig_unw.entries)
run3_tback_unw, run3_fback_unw = get_formatted_root_tree(os.path.join(tau_data_directory(), 'back_ntuple_turnonrunIII.root'))
print(run3_tback_unw.entries)

In [ ]:
low_cut_unw = 11.9
run3_sig_pt_low_unw = TH1F('hist', 'Run-III True Tau Pt (Reconstructed Et > {} GeV)'.format(low_cut_unw), 100, 0, 100)
run3_sig_pt_low_unw.GetXaxis().SetTitle('True Tau Pt')
run3_sig_pt_low_unw.GetYaxis().SetTitle('Entries')

In [ ]:
for i in range(run3_tsig_unw.entries):
    run3_tsig_unw.root_ttree.GetEntry(i)
    if run3_tsig_unw.root_ttree.Run3Et <= low_cut_unw:
        continue
    run3_sig_pt_low_unw.Fill(run3_tsig_unw.root_ttree.TrueTauPt)

In [ ]:
c1.Clear()
run3_sig_pt_low_unw.Draw()
c1.Draw()    

In [ ]:
run3_turnon_low_unw = TH1F('hist', 'Run-III Turn-On (Reconstructed Et > {} GeV)'.format(low_cut_unw), 100, 0, 100)
run3_turnon_low_unw.GetXaxis().SetTitle('True Tau Pt')
run3_turnon_low_unw.GetYaxis().SetTitle('Event Fraction')

In [ ]:
for i in range(run3_tsig_unw.entries):
    run3_tsig_unw.root_ttree.GetEntry(i)
    if run3_tsig_unw.root_ttree.Run3Et <= low_cut_unw:
        continue
    pt = run3_tsig_unw.root_ttree.TrueTauPt
    for i in range(100):
        if pt < i:
            run3_turnon_low_unw.Fill(i)
run3_turnon_low_unw.Scale(1/23448)

In [ ]:
c1.Clear()
run3_turnon_low_unw.Draw('hist')
run3_turnon_low_unw.SetStats(0)
c1.Draw()

In [ ]:
run3_sig_pt_high_unw = TH1F('hist', 'Run-III True Tau Pt (Reconstructed Et > {} GeV)'.format(high_cut_unw), 100, 0, 100)
run3_sig_pt_high_unw.GetXaxis().SetTitle('True Tau Pt')
run3_sig_pt_high_unw.GetYaxis().SetTitle('Entries')

In [ ]:
for i in range(run3_tsig_unw.entries):
    run3_tsig_unw.root_ttree.GetEntry(i)
    if run3_tsig_unw.root_ttree.Run3Et <= high_cut_unw:
        continue
    run3_sig_pt_high_unw.Fill(run3_tsig_unw.root_ttree.TrueTauPt)

In [ ]:
c1.Clear()
run3_sig_pt_high_unw.Draw()
c1.Draw()    

In [ ]:
run3_turnon_high_unw = TH1F('hist', 'Run-III Turn-On (Reconstructed Et > {} GeV)'.format(high_cut_unw), 100, 0, 100)
run3_turnon_high_unw.GetXaxis().SetTitle('True Tau Pt')
run3_turnon_high_unw.GetYaxis().SetTitle('Event Fraction')

In [ ]:
for i in range(run3_tsig_unw.entries):
    run3_tsig_unw.root_ttree.GetEntry(i)
    if run3_tsig_unw.root_ttree.Run3Et <= high_cut_unw:
        continue
    pt = run3_tsig_unw.root_ttree.TrueTauPt
    for i in range(100):
        if pt < i:
            run3_turnon_high_unw.Fill(i)
run3_turnon_high_unw.Scale(1/12533)

In [ ]:
c1.Clear()
run3_turnon_high_unw.Draw('hist')
run3_turnon_high_unw.SetStats(0)
c1.Draw()

In [ ]:
c1.Clear()
run3_turnon_low.Draw('hist')
run3_turnon_low.SetLineColor(kBlue)
run3_turnon_low.SetStats(0)
run3_turnon_low.SetTitle('Run-III Turn-On Weighted vs. Unweighted')
run3_turnon_high.Draw('hsame')
run3_turnon_high.SetLineColor(kRed)
run3_turnon_low_unw.Draw('hsame')
run3_turnon_low_unw.SetLineColor(kGreen)
run3_turnon_high_unw.Draw('hsame')
run3_turnon_high_unw.SetLineColor(kMagenta)

l1 = TLegend(0.55, 0.1, 0.9, 0.35)
l1.AddEntry(run3_turnon_low, 'Run-III Weighted @ 16.725 GeV')
l1.AddEntry(run3_turnon_high, 'Run-III Weighted @ 28.45 GeV')
l1.AddEntry(run3_turnon_low_unw, 'Run-III Unweighted @ 11.9 GeV')
l1.AddEntry(run3_turnon_high_unw, 'Run-III Unweighted @ 20.1 GeV')
l1.Draw('same')
c1.Draw()
#c1.Print('../../plots/TurnOnWeightedUnweighted.pdf')